<a href="https://colab.research.google.com/github/AliTavakoli2001/Deep-Learning-Project/blob/main/Variational%20Autoencoder%20(VAE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt

# Load and preprocess MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), -1))
x_test = x_test.reshape((len(x_test), -1))

# Define VAE architecture
latent_dim = 2  # Dimension of the latent space
input_shape = x_train.shape[1]

# Encoder
inputs = Input(shape=(input_shape,))
h = Dense(256, activation='relu')(inputs)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])
encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')

# Decoder
decoder_input = Input(shape=(latent_dim,))
decoder_h = Dense(256, activation='relu')(decoder_input)
decoder_output = Dense(input_shape, activation='sigmoid')(decoder_h)
decoder = Model(decoder_input, decoder_output, name='decoder')

# VAE Model
outputs = decoder(z)
vae = Model(inputs, outputs, name='vae')

# VAE Loss
reconstruction_loss = binary_crossentropy(inputs, outputs) * input_shape
kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam')

# Train the VAE
history = vae.fit(x_train, epochs=50, batch_size=256, validation_data=(x_test, None))

# Generate new images
encoded_imgs, _, _ = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

# Visualization
def plot_results(x_test, decoded_imgs, n=10):
    plt.figure(figsize=(20, 4))
    for i in range(n):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(x_test[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
        if i == 0:
            ax.set_title("Test Images")

        ax = plt.subplot(2, n, i + 1 + n)
        plt.imshow(decoded_imgs[i].reshape(28, 28), cmap='gray')
        plt.axis('off')
        if i == 0:
            ax.set_title("Decoded Images")
    plt.show()

plot_results(x_test, decoded_imgs)
